In [ ]:
#importing dependencies
import torch
import torch.nn as nn #neural network
import torch.optim as optim #to load optimizers
from sklearn.datasets import load_breast_cancer
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

Using device: cuda


In [ ]:
#loading dataset
data = load_breast_cancer()
x = data.data
y = data.target

In [ ]:
print(x)

[[1.799e+01 1.038e+01 1.228e+02 ... 2.654e-01 4.601e-01 1.189e-01]
 [2.057e+01 1.777e+01 1.329e+02 ... 1.860e-01 2.750e-01 8.902e-02]
 [1.969e+01 2.125e+01 1.300e+02 ... 2.430e-01 3.613e-01 8.758e-02]
 ...
 [1.660e+01 2.808e+01 1.083e+02 ... 1.418e-01 2.218e-01 7.820e-02]
 [2.060e+01 2.933e+01 1.401e+02 ... 2.650e-01 4.087e-01 1.240e-01]
 [7.760e+00 2.454e+01 4.792e+01 ... 0.000e+00 2.871e-01 7.039e-02]]


In [ ]:
print(y[:5])

[0 0 0 0 0]


In [ ]:
#splitting into test and train
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.2, random_state = 42)

In [ ]:
print(x.shape)
print(x_train.shape)
print(x_test.shape)

(569, 30)
(455, 30)
(114, 30)


In [ ]:
#standardizing data using Standardscaler
scaler = StandardScaler()

x_train = scaler.fit_transform(x_train)
x_test = scaler.transform(x_test)

In [ ]:
type(x_train)

numpy.ndarray

In [ ]:
#converting data to pytorch tensors & move it to GPU
x_train = torch.tensor(x_train, dtype = torch.float32).to(device)
y_train = torch.tensor(y_train, dtype = torch.float32).to(device)
x_test = torch.tensor(x_test, dtype = torch.float32).to(device)
y_test = torch.tensor(y_test, dtype = torch.float32).to(device)

In [ ]:
#defining nn architecture
class NeuralNet(nn.Module):

  def __init__(self, input_size, hidden_size, output_size):
    super(NeuralNet, self).__init__()
    self.fc1 = nn.Linear(input_size, hidden_size)
    self.relu = nn.ReLU()
    self.fc2 = nn.Linear(hidden_size, output_size)
    self.sigmoid = nn.Sigmoid()

  def forward(self, x):
    out = self.fc1(x)
    out = self.relu(out)
    out = self.fc2(out)
    out = self.sigmoid(out)
    return out

In [ ]:
#defining hyperpaarmeters
input_size = x_train.shape[1]
hidden_size = 64
output_size = 1
learning_rate = 0.001
num_epochs = 100

In [ ]:
#Initializing nn & moving it to GPU
model = NeuralNet(input_size, hidden_size, output_size).to(device)

In [ ]:
#defining loss & optimizor
criterion = nn.BCELoss()
optimizer = optim.Adam(model.parameters(), lr = learning_rate)

In [ ]:
#Training the model
for epoch in range(num_epochs):
  model.train()
  optimizer.zero_grad()
  outputs = model(x_train)
  loss = criterion(outputs, y_train.view(-1, 1))
  loss.backward()
  optimizer.step()

  #calculating accuracy
  with torch.no_grad():
      predicted = outputs.round()
      correct = (predicted == y_train.view(-1, 1)).float().sum()
      accuracy = correct / y_train.size(0)

      if (epoch + 1) % 10 == 0:
        print(f"Epoch [{epoch + 1}/{num_epochs}], Loss : {loss.item():.4f}, Accuracy : {accuracy.item() * 100:.2f}%")


Epoch [10/100], Loss : 0.5488, Accuracy : 86.59%
Epoch [20/100], Loss : 0.4445, Accuracy : 89.89%
Epoch [30/100], Loss : 0.3604, Accuracy : 91.65%
Epoch [40/100], Loss : 0.2932, Accuracy : 92.53%
Epoch [50/100], Loss : 0.2412, Accuracy : 93.85%
Epoch [60/100], Loss : 0.2020, Accuracy : 95.60%
Epoch [70/100], Loss : 0.1728, Accuracy : 96.04%
Epoch [80/100], Loss : 0.1510, Accuracy : 96.26%
Epoch [90/100], Loss : 0.1342, Accuracy : 96.92%
Epoch [100/100], Loss : 0.1210, Accuracy : 97.14%


In [ ]:
#evaluating on training set
model.eval()
with torch.no_grad():
  outputs = model(x_train)
  predicted = outputs.round()
  correct = (predicted == y_train.view(-1, 1)).float().sum()
  accuracy = correct / y_train.size(0)
  print(f"Accuracy on training data: {accuracy.item() * 100:.2f}%")

Accuracy on training data: 97.36%


In [ ]:
#evaluating on training set
model.eval()
with torch.no_grad():
  outputs = model(x_test)
  predicted = outputs.round()
  correct = (predicted == y_test.view(-1, 1)).float().sum()
  accuracy = correct / y_test.size(0)
  print(f"Accuracy on test data: {accuracy.item() * 100:.2f}%")

Accuracy on test data: 97.37%


In [ ]:
import pickle